In [6]:
import requests
import re
from bs4 import BeautifulSoup
import collections
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import collections

In [21]:
from pymystem3 import Mystem
m = Mystem()

# Задание 1

Соберем 30 положительных отзывов о баре "Ровесник" с яндекс карт.

In [7]:
def find_positive_reviews():
    result = requests.get('https://yandex.ru/maps/org/rovesnik/168523045237/reviews/?ll=37.605949%2C55.762399&z=15')
    html = result.text
    positive_reviews = []
    soup = BeautifulSoup(html,'html.parser')
    review = soup.find_all('div', {'class': 'reviews-view__reviews'})
    for rev in soup.find_all('div', {'class': 'business-review-view'}):
        counter = 0
        text = rev.find('div', {'class': 'business-review-view__body-text _collapsed'})
        t = text.get_text()
        for stars in rev.find_all('span', {'class': 'business-rating-badge-view__star _size_m'}):
            counter += 1
            if counter == 5 and len(positive_reviews) < 30:
                positive_reviews.append(t) 
    return positive_reviews

Так как в яндекс картах, чтобы открыть больше отзывов, нужно нажать кнопку "показать еще" (то есть я не могу так поменять ссылку, чтобы увидеть больше отзывов), то я просто пройдусь одной функцией по трем разным барам, чтобы набрать 30 отрицательных отзывов. 

In [8]:
def find_negative_reviews(negative_reviews, link):
    result = requests.get(link)
    html = result.text
    soup = BeautifulSoup(html,'html.parser')
    review = soup.find_all('div', {'class': 'reviews-view__reviews'})
    for rev in soup.find_all('div', {'class': 'business-review-view'}):
        counter = 0
        text = rev.find('div', {'class': 'business-review-view__body-text _collapsed'})
        t = text.get_text()
        for stars in rev.find_all('span', {'class': 'business-rating-badge-view__star _empty _size_m'}):
            counter += 1
            if counter == 3 and len(negative_reviews) < 30:
                negative_reviews.append(t)
    return negative_reviews

# Задание 2

Отчистим положительные отзывы от пунктуации, приведем слова к нижнему регистру, токенизируем каждый отзыв, приведем каждый токен к нормальной форме и запишем в список. 

In [28]:
def tokenize_positive_reviews(positive_reviews):
    tokenized_positive = []
    for text in positive_reviews:
        words = re.sub(r'[^\w^\s\']', '', text)
        words = words.lower()
        tokenized_text = word_tokenize(words)
        for word in tokenized_text:
            ana = morph.parse(word)
            ana = ana[0]
            lemma = ana.normal_form
            tokenized_positive.append(lemma)
    return tokenized_positive

# Функция из дз 2

In [51]:
def positive_sintactic_groups(positive_reviews, tokenized_positive):
    words = []
    pos = []
    for text in positive_reviews:
        ana = m.analyze(text)
        for a in ana:
            if ('analysis' in a) and (a['analysis'] != []):
                gr = a['analysis'][0]['gr']
                pos_tag = gr.split('=')[0].split(',')[0]
                word = a['text']
                words.append(word)
                pos.append(pos_tag)
        for i in range(1, len(pos)):
            if pos[i] == 'V' and pos[i-1] == 'ADVPRO':
                group = words[i] + ' ' + words[i-1]
                tokenized_positive.append(group)
            elif pos[i] == 'ADVPRO' and pos[i-1] == 'V':
                group = words[i] + ' ' + words[i-1]
                tokenized_positive.append(group)
            elif pos[i] == 'S' and pos[i-1] == 'A':
                group = words[i] + ' ' + words[i-1]
                tokenized_positive.append(group)
    return tokenized_positive

Здесь сделаем все то же самое, только для отрицательных отзывов. 

In [34]:
def tokenize_negative_reviews(negative_reviews):
    tokenized_negative = []
    for text in negative_reviews:
        words = re.sub(r'[^\w^\s\']', '', text)
        words = words.lower()
        tokenized_text = word_tokenize(words)
        for word in tokenized_text:
            ana = morph.parse(word)
            ana = ana[0]
            lemma = ana.normal_form
            tokenized_negative.append(lemma)
    return tokenized_negative

# Функция из дз 2

In [54]:
def negative_sintactic_groups(negative_reviews, tokenized_negative):
    words = []
    pos = []
    for text in negative_reviews:
        ana = m.analyze(text)
        for a in ana:
            if ('analysis' in a) and (a['analysis'] != []):
                gr = a['analysis'][0]['gr']
                pos_tag = gr.split('=')[0].split(',')[0]
                word = a['text']
                words.append(word)
                pos.append(pos_tag)
        for i in range(1, len(pos)):
            if pos[i] == 'V' and pos[i-1] == 'ADVPRO':
                group = words[i] + ' ' + words[i-1]
                tokenized_negative.append(group)
            elif pos[i] == 'ADVPRO' and pos[i-1] == 'V':
                group = words[i] + ' ' + words[i-1]
                tokenized_negative.append(group)
            elif pos[i] == 'S' and pos[i-1] == 'A':
                group = words[i] + ' ' + words[i-1]
                tokenized_negative.append(group)
    return tokenized_negative

# Задание 3

Составим два словаря уникальных слов (положительных и отрицательных), где ключомами является слова, а значениями - то, сколько раз они встретились. 
Затем создадим два списка, для положительных и отрицательных слов, встретившихся больше двух раз. 

In [13]:
def clean_words(tokenized_positive, tokenized_negative):
    positive_words = []
    positive = []
    for x in tokenized_positive:
        if x not in tokenized_negative:
            positive_words.append(x)
    counter = collections.Counter(positive_words)
    counter = dict(counter)
    for key in counter:
        if counter[key] != 1 and counter[key] != 2:
            positive.append(key)
    negative_words = []
    negative = []
    for x in tokenized_negative:
        if x not in tokenized_positive:
            negative_words.append(x)
    counter_2 = collections.Counter(negative_words)
    counter_2 = dict(counter_2)
    for key in counter_2:
        if counter_2[key] != 1 and counter_2[key] != 2:
            negative.append(key)
    return positive, negative

# Задание 4

Следующая функция на вход принимает текст, из которого создает список нормальных форм. Затем она проходится по каждому слову и считает, сколько из них встречаются в списках с положительными и отрицательными словами, и возвращает ответ.

In [71]:
def detect(positive, negative, text):
    tokenized_review = []
    words = re.sub(r'[^\w^\s\']', '', text)
    words = words.lower()
    tokenized_text = word_tokenize(words)
    for word in tokenized_text:
        ana = morph.parse(word)
        ana = ana[0]
        lemma = ana.normal_form
        tokenized_review.append(lemma)
    counter_pos = 0
    counter_neg = 0
    for lemma in tokenized_review:
        if lemma in positive:
            counter_pos += 1
        if lemma in negative:
            counter_neg += 1
    if counter_pos > counter_neg:
        return 'pos'
    else:
        return 'neg'

In [25]:
!pip install sklearn

In [15]:
from sklearn.metrics import accuracy_score

Считаем, с какой точностью наша программа определяет тональность. 

In [66]:
def test_positive():
    positive_test = []
    result = requests.get('https://yandex.ru/maps/org/ryumochnaya_svoboda/147745010375/reviews/?ll=37.637587%2C55.762414&z=15')
    html = result.text
    positive_reviews = []
    soup = BeautifulSoup(html,'html.parser')
    review = soup.find_all('div', {'class': 'reviews-view__reviews'})
    for rev in soup.find_all('div', {'class': 'business-review-view'}):
        counter = 0
        text = rev.find('div', {'class': 'business-review-view__body-text _collapsed'})
        t = text.get_text()
        for stars in rev.find_all('span', {'class': 'business-rating-badge-view__star _size_m'}):
            counter += 1
            if counter == 5 and len(positive_reviews) < 30:
                positive_test.append(t) 
    return positive_test

In [67]:
def test_negative():
    negative_test = []
    result = requests.get('https://yandex.ru/maps/213/moscow/?display-text=KillFish&ll=37.648735%2C55.759693&mode=search&oid=1902978412&ol=biz&sll=37.631343%2C55.742509&sspn=0.008154%2C0.007469&text=%7B%22text%22%3A%22KillFish%22%2C%22what%22%3A%5B%7B%22attr_name%22%3A%22chain_id%22%2C%22attr_values%22%3A%5B%222778132496%22%5D%7D%5D%7D&z=13')
    html = result.text
    positive_reviews = []
    soup = BeautifulSoup(html,'html.parser')
    review = soup.find_all('div', {'class': 'reviews-view__reviews'})
    for rev in soup.find_all('div', {'class': 'business-review-view'}):
        counter = 0
        text = rev.find('div', {'class': 'business-review-view__body-text _collapsed'})
        t = text.get_text()
        for stars in rev.find_all('span', {'class': 'business-rating-badge-view__star _empty _size_m'}):
            counter += 1
            if counter == 3 and len(negative_reviews) < 30:
                negative_test.append(t)
    return negative_test

In [68]:
def test_detect(positive, negative):
    results = []
    gold = []
    for text in test_positive():
        predicted = detect(positive, negative, text)
        results.append(predicted)
        gold.append('pos')
    for text in test_negative():
        predicted = detect(positive, negative, text)
        results.append(predicted)
        gold.append('neg')
    #print(results) 
    #print(gold)
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [69]:
def main():
    negative_reviews = []
    link_1 = 'https://yandex.ru/maps/org/rovesnik/168523045237/reviews/?ll=37.605949%2C55.762399&z=15'
    link_2 = 'https://yandex.ru/maps/org/profsoyuz_barnoye_obyedineniye/133933857803/reviews/?ll=37.600092%2C55.782333&z=17'
    link_3 = 'https://yandex.ru/maps/org/zinziver/27339938644/reviews/?ll=37.645473%2C55.758883&z=14'
    a = find_positive_reviews()
    b = find_negative_reviews(negative_reviews, link_1)
    c = find_negative_reviews(b, link_2)
    d = find_negative_reviews(c, link_3)
    q = tokenize_positive_reviews(a)
    m = positive_sintactic_groups(a, q)
    w = tokenize_negative_reviews(d)
    n = negative_sintactic_groups(d, w)
    e, r = clean_words(m, n)
    t = test_detect(e, r)

# Как мы видим, по сравнению с прошлой тетрадкой точность стала выше  

In [72]:
main()

Accuracy: 0.7059


# Задание 5

1) Понятно, что нужно увеличить количество данных;
2) Можно также из отзывов поубирать стоп слова, чтобы очистить множества от слов, не несущих эмоциональную окраску;
3) Можно добавить в алгоритм векторайзер, чтобы потом сравнивать ветора с помощью разных метрик;
4) Или можно использовать word2vec, чтобы искать семантически близкие слова;